<a href="https://colab.research.google.com/github/kzon94/owl-quiz-v2/blob/main/Owl-Quiz-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import csv
import os
import getpass
import matplotlib.pyplot as plt

MIN_VALUE = 0
MAX_VALUE = 1000
DIFICULTADES = {"Fácil": 20, "Medio": 12, "Difícil": 5}
RUTA_CSV = "/content/estadisticas.csv"

def crear_directorio_si_no_existe(ruta):
    if not os.path.exists(ruta):
        os.makedirs(ruta)

crear_directorio_si_no_existe(os.path.dirname(RUTA_CSV))

def guardar_partida_csv(datos):
    existe = os.path.isfile(RUTA_CSV)
    with open(RUTA_CSV, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        if not existe:
            writer.writerow(["Nombre", "Modo", "Dificultad", "NumeroObjetivo", "Distancias", "IntentosUsados", "Resultado"])
        dist_str = "|".join(map(str, datos['distancias']))
        writer.writerow([
            datos['nombre'],
            datos['modo'],
            datos['dificultad'],
            datos['numero'],
            dist_str,
            datos['intentos_usados'],
            datos['resultado']
        ])

def leer_partidas_csv():
    partidas = []
    if not os.path.isfile(RUTA_CSV):
        return partidas
    with open(RUTA_CSV, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            dist_str = row["Distancias"] or ""
            dist_list = list(map(int, dist_str.split("|"))) if dist_str else []
            partidas.append({
                "Nombre": row["Nombre"],
                "Modo": row["Modo"],
                "Dificultad": row["Dificultad"],
                "Numero": int(row["NumeroObjetivo"]),
                "Distancias": dist_list,
                "IntentosUsados": int(row["IntentosUsados"]),
                "Resultado": row["Resultado"]
            })
    return partidas

def pedir_dificultad():
    while True:
        print("\n✨ Las sendas de la sabiduría:\n")
        print("1. Sereno: 20 oportunidades para iluminar tu mente")
        print("2. Moderado: 12 oportunidades para descubrir verdades")
        print("3. Profundo: 5 intentos para conectar con el cosmos\n")
        op = input("Escoge tu senda, joven aprendiz: ")
        if op == '1':
            return "Fácil", DIFICULTADES["Fácil"]
        elif op == '2':
            return "Medio", DIFICULTADES["Medio"]
        elif op == '3':
            return "Difícil", DIFICULTADES["Difícil"]
        else:
            print("\n⚠️ Ese sendero no existe en la crónica de la sabiduría.\n")

def partida_solitario():
    dificultad, intentos_max = pedir_dificultad()
    numero_obj = random.randint(MIN_VALUE+1, MAX_VALUE)
    nombre_jugador = input("\n🧙‍♂️🧙‍♀️ Revela tu identidad, alma sabia: ").strip() or "Anónimo"
    distancias = []
    intentos = 0

    while intentos < intentos_max:
        try:
            intento = int(input(f"\n💡 Sabiduría en acción: intento {intentos+1} de {intentos_max} (entre {MIN_VALUE+1} y {MAX_VALUE}): "))
        except:
            print("\n🚫 Oh, ¿Acaso las constelaciones se han desordenado? Ingresa un número válido.")
            continue
        if not (MIN_VALUE+1 <= intento <= MAX_VALUE):
            print(f"\n⚠️ Tu predicción se desvía del sendero. Escoge un número entre {MIN_VALUE+1} y {MAX_VALUE}.")
            continue

        intentos += 1
        dist = abs(intento - numero_obj)
        distancias.append(dist)

        if intento == numero_obj:
            print("\n🎯 En el silencio de la noche, la verdad se revela: ¡acertaste, alma sabia! 🦉\n")
            guardar_partida_csv({
                'nombre': nombre_jugador,
                'modo': 'Solitario',
                'dificultad': dificultad,
                'numero': numero_obj,
                'distancias': distancias,
                'intentos_usados': intentos,
                'resultado': 'GANA'
            })
            return
        else:
            if intento < numero_obj:
                print("\n🔼 El eco del viento susurra: busca un número mayor.")
            else:
                print("\n🔽 La brisa canta: intenta con un número menor.")

    print(f"\n🔮 El tiempo se ha agotado y el misterio ha cambiado. El número secreto era {numero_obj}.\n")
    guardar_partida_csv({
        'nombre': nombre_jugador,
        'modo': 'Solitario',
        'dificultad': dificultad,
        'numero': numero_obj,
        'distancias': distancias,
        'intentos_usados': intentos,
        'resultado': 'PIERDE'
    })

def partida_dos_jugadores():
    dificultad, intentos_max = pedir_dificultad()
    print("\n🐉 Sabio Guardián, Jugador 1:")
    nombre_jugador1 = input("\nEscoge tu nombre, custodio del secreto: ").strip() or "Jugador1"

    while True:
        try:
            numero_str = getpass.getpass(f"\nRevela en silencio el número sagrado (entre {MIN_VALUE+1} y {MAX_VALUE}): ")
            numero_obj = int(numero_str)
            if MIN_VALUE+1 <= numero_obj <= MAX_VALUE:
                break
            else:
                print("\n⚠️ La cifra se aparta del camino correcto. Intenta otro número.")
        except:
            print("\n🚫 El Oráculo de los Números no comprende esa entrada.")

    print("\n🤴 Valiente Buscador, Jugador 2:")
    nombre_jugador2 = input("\nRevela tu nombre, intrépido adivino: ").strip() or "Jugador2"
    distancias = []
    intentos = 0

    while intentos < intentos_max:
        try:
            intento = int(input(f"\n💡 Sabiduría en acción: intento {intentos+1} de {intentos_max} (entre {MIN_VALUE+1} y {MAX_VALUE}): "))
        except:
            print("\n🚫 Oh, las estrellas confabulan; ingresa un número claro y preciso.")
            continue
        if not (MIN_VALUE+1 <= intento <= MAX_VALUE):
            print(f"\n⚠️ Tu predicción se desvía del sendero. Debe ser entre {MIN_VALUE+1} y {MAX_VALUE}.")
            continue

        intentos += 1
        dist = abs(intento - numero_obj)
        distancias.append(dist)

        if intento == numero_obj:
            print(f"\n🎯 Con la visión del búho, {nombre_jugador2} ha descifrado el enigma. 🦉\n")
            guardar_partida_csv({
                'nombre': nombre_jugador2,
                'modo': '2 Jugadores',
                'dificultad': dificultad,
                'numero': numero_obj,
                'distancias': distancias,
                'intentos_usados': intentos,
                'resultado': 'GANA'
            })
            return
        else:
            if intento < numero_obj:
                print("\n🔼 El eco del viento susurra: busca un número mayor.")
            else:
                print("\n🔽 La brisa canta: intenta con un número menor.")

    print(f"\n🔮 El misterio permanece y el tiempo se desvanece. El número sagrado era {numero_obj}.\n")
    guardar_partida_csv({
        'nombre': nombre_jugador2,
        'modo': '2 Jugadores',
        'dificultad': dificultad,
        'numero': numero_obj,
        'distancias': distancias,
        'intentos_usados': intentos,
        'resultado': 'PIERDE'
    })

def mostrar_estadisticas():
    partidas = leer_partidas_csv()
    if not partidas:
        print("\n💡 La sabiduría aún no ha sido registrada. Juega para dejar tu huella.")
        return

    print("\n📜 Partidas registradas:")
    for idx, p in enumerate(partidas):
        print(f"{idx}. {p['Nombre']} | {p['Modo']} | {p['Dificultad']} | Intentos: {p['IntentosUsados']} | Resultado: {p['Resultado']} | Número: {p['Numero']}")

    while True:
        sel = input("\nIngresa el número de la partida para ver su gráfico (o 'salir' para volver): ")
        if sel.lower() == 'salir':
            return
        try:
            idx = int(sel)
            if 0 <= idx < len(partidas):
                break
            else:
                print("⚠️ Número fuera de rango. Intenta de nuevo.")
        except ValueError:
            print("⚠️ Entrada inválida. Intenta de nuevo.")

    partida = partidas[idx]
    dist_list = partida['Distancias']
    if not dist_list:
        print("No hay datos de distancias para esta partida.")
        return

    plt.figure(figsize=(10, 6))
    plt.bar(range(1, len(dist_list)+1), dist_list)
    plt.title(f"Distancia al número objetivo en cada intento - {partida['Nombre']}")
    plt.xlabel("Número de intento")
    plt.ylabel("Distancia")
    plt.xticks(range(1, len(dist_list)+1))
    plt.grid(True)
    plt.show(block=False)
    plt.pause(0.1)

def menu_principal():
    while True:
        print("\n=== EL SALÓN DEL BÚHO SABIO ===\n")
        print("🦉 Bienvenido, joven aprendiz, a este templo del enigma numérico.")
        print("Aquí, cada elección te acerca o te aleja de la sabiduría:\n")
        print("- 🛤️ En el 'Sendero Solitario' descubrirás secretos por ti mismo.")
        print("- 🤝 En 'Diálogo de Sabiduría' compartirás el misterio con otro buscador.")
        print("- 📜 En 'Consulta los Pergaminos' revelarás las huellas de tus pasadas exploraciones.\n")
        print("🐉 Decide tu senda con cuidado...\n")
        print("1. Sendero Solitario")
        print("2. Diálogo de Sabiduría (2 Jugadores)")
        print("3. Consulta los Pergaminos (estadísticas)")
        print("4. Abandona el Templo\n")
        opcion = input("Elige tu senda: ")
        if opcion == '1':
            partida_solitario()
        elif opcion == '2':
            partida_dos_jugadores()
        elif opcion == '3':
            mostrar_estadisticas()
        elif opcion == '4':
            print("\n🌙 Que la noche te guíe hasta nuestro próximo encuentro.")
            break
        else:
            print("\n🚫 El destino no reconoce esa opción, elige sabiamente.")

if __name__ == "__main__":
    menu_principal()